### Requirement Libraries

In [61]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [123]:
#we can use internal openCV face detector model in version >= 3.3
cv2.__version__

'4.9.0'

In [137]:
a = np.array([3, 15, 6, 7, 6])
p = np.array([2, 12, 8, 3, 6])
N = [3, 12, 6, 7, 6]

In [138]:
tf.reduce_sum(tf.square(a - p), axis=-1)

<tf.Tensor: shape=(), dtype=int32, numpy=30>

In [139]:
tf.reduce_sum(tf.square(a - N), axis=-1)

<tf.Tensor: shape=(), dtype=int32, numpy=9>